In [181]:
import numpy as np
import scipy.io
import math

In [182]:
#get matlab data as nested dictionary
training = scipy.io.loadmat('ECG_train.mat')
testing = scipy.io.loadmat('ECG_test.mat')

#extract data from dictionaries
X_normal = training['X_train_normal']
X_abnormal = training['X_train_abnormal']

X_normal_test = testing['X_test_normal']
X_abnormal_test = testing['X_test_abnormal']
print(X_normal[1].shape)

(96,)


In [183]:
k = 7 # number of neighbors to consider
T_MAX = 1000
alpha0 = 0.9

In [184]:
def sample_func(X,t):
    index = t%X.shape[0]
    return X[index]

In [185]:
def euclidian_distance(x1,x2):
    return abs(np.linalg.norm(x1-x2))

def getLearningRate(t, alpha0, beta):
    return alpha0 * math.exp(-t*beta)

distance = euclidian_distance

In [186]:
def getBMU(prototypes, sample, distance):
    #init variables
    best_dist = float("inf")
    best_vec = 0

    #check every prototype vector
    index = 0
    bmu_index = 0
    for proto in prototypes:
        #check if distance between sample and prototype < current lowest
        new_dist = distance(sample,proto)
        if  new_dist < best_dist:
            best_dist = new_dist
            best_vec = proto
            bmu_index = index
        index += 1
    
    return bmu_index, best_vec


In [187]:
prototypes = [None] * 2

for i in range(0,2):
    X = []
    if(i == 1):
        X = X_normal
    else:
        X = X_abnormal
    #get center of data
    avg = np.zeros((1,X[0].shape[0]))
    total = 0
    #take sum of all vectors
    for x in X:
        avg += x
        total += 1
    #compute average
    avg = avg * 1/total
    prototypes[i] = np.tile(avg, (k, 1) ) + (0.5*np.ones((k, X[0].shape[0])) - np.random.rand(k, X[0].shape[0]))
    beta = math.log(10)/T_MAX
    #Loop through t:
    t = 0
    while t < T_MAX:
        sample = sample_func(X,t)
        bmu_index, bmu = getBMU(prototypes[i], sample, distance)
        alpha = getLearningRate(t, alpha0, beta)

        #update the best matching prototype vector
        index = 0
        prototypes[i][bmu_index-1] = bmu + alpha*(sample - bmu)
        t += 1
    #we are done!

In [188]:
#True: Normal ; False: Abnormal
def getClassification(sample):
    i1, bmu0 = getBMU(prototypes[0], sample, distance)
    i2, bmu1 = getBMU(prototypes[1], sample, distance)
    if(distance(sample, bmu0) < distance(sample, bmu1)):
        return True 
    else:
        return False

In [189]:
confusion_matrix = np.zeros((2,2))

#Classifying:
for x in X_normal_test:
    if getClassification(x):
        confusion_matrix[0][0] += 1
    else:
        confusion_matrix[1][0] += 1

for x in X_abnormal_test:
    if getClassification(x):
        confusion_matrix[0][1] += 1
    else:
        confusion_matrix[1][1] += 1
print(confusion_matrix)

[[16. 28.]
 [48.  8.]]
